In [23]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import numpy as np
import requests
import re
#from helpers.integration_helpers import canoicalize_smiles

In [43]:
def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

def get_smiles(compound):
    print(compound)
    smilesUrl = "https://opsin.ch.cam.ac.uk/opsin/"+compound+".smi"
    requestSmiles = requests.get(smilesUrl)
    if requestSmiles.status_code == 400 or requestSmiles.status_code == 404:
        return None
    else:
        m = Chem.MolFromSmiles(requestSmiles.text)
        if m != None:
            return Chem.MolToSmiles(Chem.MolFromSmiles(requestSmiles.text)) 
        else:
            return None
    
def celsiusToKelvin(temp):
    return temp + 273.15

def get_compound_with_element(data, element):
    return data[data['smiles'].str.contains(element)]

In [50]:
integrated_data = pd.read_csv('../data/integrated_dataset.csv')
#print(integrated_data['source'].unique())
#integrated_data[integrated_data['smiles'].str.contains('Si', regex=False)]['smiles'].unique().shape
#integrated_data.head()
#integrated_data.drop(columns=['Unnamed: 0'], inplace=True)
#list(integrated_data)
#integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

## Carroll

In [ ]:
carroll = pd.read_csv('../data/Carroll11.csv', encoding = "ISO-8859-1")

In [ ]:
print(list(carroll))
#carroll = data[['Compound', 'TFP(K)Ref.']]
#carroll['source'] = 'carroll11'
#carroll['pure substance'] = 1
carroll.head()

## Saldana

In [ ]:
fuels = pd.read_csv('../data/fuels.csv', encoding = "ISO-8859-1")
fuels.head()

## Pan12

In [ ]:
pan12 = pd.read_csv('../data/pan12.csv', encoding = "ISO-8859-1")
pan12.head()

## chen14

In [ ]:
chen14 = pd.read_csv('../data/chen14.csv', encoding = "ISO-8859-1")

In [ ]:
chen14.head()

In [ ]:
# canonicalize carroll smiles
for i in range(0, len(carroll)):
    carroll.iloc[i]['smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(carroll.iloc[i]['smiles']))


In [ ]:
#canonicalize smiles with rdkit
result['canonical smiles'] = None
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.iloc[idx]['smiles']))

In [ ]:
for idx, row in result.iterrows():
    result.loc[idx,'canonical smiles'] = Chem.MolToSmiles(Chem.MolFromSmiles(result.loc[idx]['smiles']))

## Add Gelest to dataset

In [ ]:
silanes = pd.read_csv('../data/silanes_all.csv')
metal_organic = pd.read_csv('../data/metal-organics_all.csv')
tin = pd.read_csv('../data/tin_all.csv')
germanium  = pd.read_csv('../data/germanium_all.csv')
print(list(silanes))
print(list(metal_organic))
print(list(tin))
print(list(germanium))

In [ ]:
#result = result[['compound', 'smiles', 'flashpoint', 'pure substance', 'source']]
silanes_pure = silanes[silanes['pure substance'] == 1]
metal_organic_pure = metal_organic[metal_organic['pure substance'] == 1]
tin_pure = tin[tin['pure substance'] == 1]
germanium_pure = germanium[germanium['pure substance'] == 1]

## pubchem integration

In [ ]:
pubchem = pd.read_csv('../data/pubchem_data.csv')
pubchem = pubchem[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
germanium_pure = germanium_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
tin_pure = tin_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
metal_organic_pure = metal_organic_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
silanes_pure = silanes_pure[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
hazardous_mat = hazardous_mat[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]


In [ ]:
print(list(pubchem))
print(list(carroll))
print(list(chen14))
print(list(fuels))
print(list(germanium_pure))
print(list(metal_organic_pure))
print(list(pan12))
print(list(silanes_pure))
print(list(tin_pure))
print(list(hazardous_mat))

In [ ]:
def canoicalize_smiles(data):
    for idx, row in data.iterrows():
        m = Chem.MolFromSmiles(data.iloc[idx]['smiles'])
        if m != None:
            data.iloc[idx]['smiles'] = Chem.MolToSmiles(m)
        else:
            data.iloc[idx]['smiles'] = None
    return data

In [ ]:
frames = [pubchem, carroll, chen14, fuels, germanium_pure, metal_organic_pure, pan12, silanes_pure, tin_pure, hazardous_mat]
result = pd.concat(frames)

In [ ]:
canonical_smiles_data = canoicalize_smiles(result)

In [ ]:
print(canonical_smiles_data.shape)
print(canonical_smiles_data.dropna().shape)
canonical_smiles_data.dropna(inplace=True)

In [ ]:
canonical_smiles_data.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
def remove_duplicates(data):
    duplicates = data[data.duplicated(subset='smiles')]
    result = data.drop_duplicates(subset='smiles', keep=False)#[~duplicates]
    #for each unique smiles that has duplicates
    for smiles in data[data.duplicated(subset='smiles')]['smiles'].unique():
        dup_rows = data.loc[data['smiles'] == smiles]
        if dup_rows['flashpoint'].unique().shape[0] == 1:
            # remove all but one
            result = result.append(dup_rows.iloc[0], sort=False)
        else:
            if dup_rows['flashpoint'].std() < 5:
                # add 1 back
                result = result.append(dup_rows.iloc[0], sort=False)
    return result  
            

In [ ]:
pure_compounds = data.loc[data['pure substance'] == 1]
pure_compounds.shape

In [ ]:
data[data.duplicated(subset='smiles')].shape

In [ ]:
data = data.loc[data['pure substance'] == 1]
print(data.shape)
no_dups = remove_duplicates(data)
print(no_dups.shape)

In [ ]:
no_dups[no_dups.duplicated(subset='smiles')]
#no_dups.loc[no_dups['smiles'] == 'CC1=CCCC(=C)C2CC(C2CC1)(C)C']
no_dups.to_csv('../data/integrated_dataset.csv', index=False)

In [ ]:
result = data.drop_duplicates(subset='smiles', keep=False)
result.shape

In [ ]:
data[data.duplicated(subset='smiles')]['smiles'].unique()

### Hazardous Materials

In [ ]:
data = pd.read_csv('../data/hazardous_materials.csv')
haz = data[['Common name', 'Data type', 'Measurement name', 'Measurement value', 'Measurement units']]

In [ ]:
flashpoints = haz.loc[haz['Measurement name'] == 'Flash point']

In [ ]:
print(list(flashpoints))

In [ ]:
flashpoints['smiles'] = flashpoints['Common name'].apply(get_smiles)

In [ ]:
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('<')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('>')]
flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('~')]
pattern = '\\w+-\\w'
flashpoints['Measurement value'] = flashpoints['Measurement value'].str.replace(pattern, 'none', regex=True)
#flashpoints = flashpoints[~flashpoints['Measurement value'].str.contains('none')]
flashpoints['Measurement value'] = pd.to_numeric(flashpoints['Measurement value'], errors='coerce')

flashpoints.head()

In [ ]:
flashpoints['flashpoint'] = flashpoints['Measurement value'].apply(celsiusToKelvin)
flashpoints.drop(columns=['Measurement name','Data type', 'Measurement units', 'Measurement value'], inplace=True)
#flashpoints.rename(columns={'Common name' : 'compound', 'Measurement value' : 'flashpoint'}, inplace=True)

In [ ]:
flashpoints.dropna(inplace=True)
flashpoints.rename(columns={'Common name' : 'compound'}, inplace=True)
#flashpoints.drop(df.columns[i], axis=1)
flashpoints['pure substance'] = 1
flashpoints['source'] = 'hazardous materials'
hazardous_mat = flashpoints[['compound', 'flashpoint', 'pure substance', 'smiles', 'source']]
hazardous_mat.shape

In [ ]:
hazardous_mat.head()

In [ ]:
#data = pd.read_csv('../data/integrated_dataset.csv')
#frames = [data, hazardous_mat]
#result = pd.concat(frames)
no_dups = remove_duplicates(result)
print(result.shape)
print(no_dups.shape)
#no_dups.to_csv('../data/integrated_dataset.csv', index=False)

## Le 2015

In [79]:
og_le15 = pd.read_csv('../data/le15.csv')
og_le15_2 = pd.read_csv('../data/le15_2.csv')
#le15.rename(columns={"MolID" : "compound", "FP" : "flashpoint"}, inplace=True)
#print(le15.head())
#print(le15_2.head())

In [71]:
le15['flashpoint'] = le15['flashpoint'].apply(celsiusToKelvin)
le15_2['flashpoint'] = le15_2['flashpoint'].apply(celsiusToKelvin)

In [72]:
le15['smiles'] = le15['compound'].apply(get_smiles)
le15_2['smiles'] = le15_2['compound'].apply(get_smiles)

In [81]:
le15['flashpoint'] = og_le15['flashpoint'].apply(celsiusToKelvin)
le15_2['flashpoint'] = og_le15_2['flashpoint'].apply(celsiusToKelvin)
le15_2.head()

,compound,flashpoint,smiles
0,(4-bromophenyl)urea,402.85,NC(=O)Nc1ccc(Br)cc1
1,[dimethyl(trimethylsilyloxy)silyl]oxy-dimethyl...,361.25,C[Si](C)(C)O[Si](C)(C)O[Si](C)(C)O[Si](C)(C)C
2,"1,1,3,3-tetraethylurea",360.15,CCN(CC)C(=O)N(CC)CC
3,"1,1,3-trimethylurea",367.55,CNC(=O)N(C)C
4,"1,2-dihydroAcenaphthalene120oC",393.15,None


In [82]:
le15_smiles = le15[le15.smiles.isnull() == False]
le15_2_smiles = le15_2[le15_2.smiles.isnull() == False]
le15_no_smiles = le15[le15.smiles.isnull() == True]
le15_2_no_smiles = le15_2[le15_2.smiles.isnull() == True]

In [204]:
def clean_string(string):
    #remove ending number
    if ',' in string:
        return string
    pat = r'\w+\D'
    prog = re.compile(pat)
    match = prog.match(string)
    if match != None:
        string = match.group(0)
    #remove [0-9]{1,2}oC
    string = re.sub(r'[0-9]{1,2}oC', '', string)
    #add , to [0-9]{2}
    pat2 = r'^[0-9]{2}'
    if re.search(pat2, string) != None:
        string = string[0] + ',' + string[1:len(string)]
    return string
    #else:
    #    return string

In [209]:
print(le15_no_smiles.shape)
le15_no_smiles['cleaned_compound'] = ''
le15_no_smiles['cleaned_compound'] = le15_no_smiles['compound'].apply(clean_string)
le15_no_smiles['smiles'] = le15_no_smiles['cleaned_compound'].apply(get_smiles)

(2073, 4)


C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [214]:
print(le15_2_no_smiles.shape)
le15_2_no_smiles['cleaned_compound'] = ''
le15_2_no_smiles['cleaned_compound'] = le15_2_no_smiles['compound'].apply(clean_string)
le15_2_no_smiles['smiles'] = le15_2_no_smiles['cleaned_compound'].apply(get_smiles)

(1483, 3)


C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [217]:
le15_2_no_smiles.dropna(inplace=True)
le15_no_smiles.dropna(inplace=True)
frames = [le15_2_no_smiles, le15_no_smiles]
le_reclaimed = pd.concat(frames)
print(le_reclaimed.shape)

(1517, 4)


C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [221]:
le_reclaimed.drop(columns=['compound'], inplace=True)
le_reclaimed.rename(columns={'cleaned_compound' : 'compound'}, inplace=True)
le_reclaimed['source'] = 'le15'
le_reclaimed['pure substance'] = 1
le_reclaimed = le_reclaimed[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [233]:
le_reclaimed#.head()
all_data[all_data['smiles'] == 'Cl[Si](Cl)(Cl)C(Br)CBr']

,smiles,compound,flashpoint,source,pure substance


In [224]:
all_data = pd.read_csv('../data/integrated_dataset.csv')
print(all_data['smiles'].unique().shape)

(10459,)


In [235]:
#frames = [all_data, le_reclaimed]
#result = pd.concat(frames)
result.smiles.unique().shape
#result.to_csv('../data/integrated_dataset.csv', index=False)

(10581,)

In [78]:
le15_smiles['source'] = 'le15'
le15_smiles['pure substance'] = 1
le15_smiles = le15_smiles[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
le15_smiles['flashpoint'] = le15_smiles['flashpoint'].apply(celsiusToKelvin) 
print(list(le15_smiles))

['smiles', 'compound', 'flashpoint', 'source', 'pure substance']


In [79]:
frames = [all_data, le15_smiles]
result = pd.concat(frames)
result.smiles.unique().shape

(10017,)

In [93]:
result['source'].unique()

array(['pubchem', 'carroll11', 'chen14', 'Saldana11', 'gelest_germanium',
       'gelest_metal-organics', 'pan12', 'gelest_silanes', 'gelest_tin',
       'hazardous materials', 'le15'], dtype=object)

In [89]:
test = result.drop_duplicates(subset='smiles', keep='first')
test.shape
test = test[['smiles', 'flashpoint']]
test.to_csv('all_fp.csv', index=False)

In [3]:
#result[result['source'] == 'le15'].head()
result = pd.read_csv('../data/integrated_dataset.csv')

### Wang11

In [6]:
wang = pd.read_csv('../data/wang11.csv')
wang.head()
print(wang.shape)

(230, 2)


In [7]:
wang['smiles'] = wang['compound'].apply(get_smiles)
wang.head()

,compound,flashpoint,smiles
0,Methyltrichlorosilane,276.15,C[Si](Cl)(Cl)Cl
1,Vinyltrichlorosilane,294.15,C=C[Si](Cl)(Cl)Cl
2,Dichloromethylmethyldichlorosilane,301.15,C[Si](Cl)(Cl)C(Cl)Cl
3,Ethyltrichlorosilane,263.15,CC[Si](Cl)(Cl)Cl
4,Dimethylchlorosilane,245.15,C[SiH](C)Cl


In [8]:
wang[wang['smiles'].isnull() == False].shape

(210, 3)

In [9]:
wang.dropna(inplace=True)
wang.shape

(210, 3)

In [11]:
wang['pure substance'] = 1
wang['source'] = 'wang11'
print(list(integrated_data))
print(list(wang))

['smiles', 'compound', 'flashpoint', 'source', 'pure substance']
['compound', 'flashpoint', 'smiles', 'pure substance', 'source']


In [12]:
wang = wang[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [14]:
frames = [integrated_data, wang]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

(10158,)

### Didier14

In [16]:
didier14 = pd.read_csv('../data/didier14.csv')
didier14.dropna(inplace=True)
print(didier14.shape)
didier14['smiles'] = didier14['Name'].apply(get_smiles)
didier14['source'] = 'didier14'
didier14['pure substance'] = 1
print(list(didier14))

(1457, 4)
['CASRN', 'Name', 'calc.', 'exp.', 'smiles', 'source', 'pure substance']


In [89]:
didier14.dropna(inplace=True)
print(didier14.shape)

(1457, 7)
(1328, 7)


In [17]:
didier14.rename(index=str, columns={'Name' : 'compound', 'exp.' : 'flashpoint'}, inplace=True)
didier14.drop(columns=['CASRN', 'calc.'])
didier14 = didier14[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [18]:
frames = [integrated_data, didier14]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

(10403,)

In [84]:
smi = Chem.MolToSmiles(Chem.MolFromSmiles('ClC(C)CCC(C)C'))
#result[result['smiles'] == smi]

In [85]:
combined[combined['smiles'] == smi]

,smiles,compound,flashpoint,source,pure substance


### Yong_Pan07

In [21]:
pan07 = pd.read_csv('../data/yong_pan07.csv')
print(pan07.shape)
print(list(pan07))

(92, 5)
['No.', 'Compound', 'Experimental FP (K)', 'Predicted FP (K)', 'Deviation (K)']


In [23]:
pan07['smiles'] = pan07['Compound'].apply(get_smiles)
pan07.dropna(inplace=True)
print(pan07.shape)
pan07['source'] = 'pan07'
pan07['pure substance'] = 1
print(list(pan07))
print(list(integrated_data))

(92, 8)
['No.', 'Compound', 'Experimental FP (K)', 'Predicted FP (K)', 'Deviation (K)', 'smiles', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']


In [24]:
pan07.rename(index=str, columns={'Compound' : 'compound', 'Experimental FP (K)' : 'flashpoint'}, inplace=True)
pan07.drop(columns=['No.', 'Predicted FP (K)', 'Deviation (K)'])
pan07 = pan07[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [25]:
frames = [integrated_data, pan07]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

(10403,)

### carroll15

In [26]:
carroll15 = pd.read_csv('../data/carroll15.csv')
print(carroll15.shape)
print(list(carroll15))

(82, 4)
['Entry', 'Compound', 'Y BP Pred.', 'T FP (K) Lit.']


In [27]:
carroll15['smiles'] = carroll15['Compound'].apply(get_smiles)
carroll15.dropna(inplace=True)
print(carroll15.shape)
carroll15['source'] = 'caroll15'
carroll15['pure substance'] = 1
print(list(carroll15))
print(list(integrated_data))

(81, 5)
['Entry', 'Compound', 'Y BP Pred.', 'T FP (K) Lit.', 'smiles', 'source', 'pure substance']
['smiles', 'compound', 'flashpoint', 'source', 'pure substance']


In [28]:
carroll15.rename(index=str, columns={'Compound' : 'compound', 'T FP (K) Lit.' : 'flashpoint'}, inplace=True)
carroll15.drop(columns=['Entry', 'Y BP Pred.'])
carroll15 = carroll15[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [29]:
frames = [integrated_data, carroll15]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

(10461,)

### Didier12

In [36]:
didier12 = pd.read_csv('../data/didier12.csv')
didier12 = didier12[['Molecule', 'Exp.(K)']]
print(didier12.shape)
print(list(didier12))

(230, 2)
['Molecule', 'Exp.(K)']


In [37]:
didier12['smiles'] = didier12['Molecule'].apply(get_smiles)
didier12.dropna(inplace=True)
print(didier12.shape)
didier12['source'] = 'didier12'
didier12['pure substance'] = 1
print(list(didier12))
print(list(integrated_data))

Methyltrichlorosilane
Vinyltrichlorosilane
Dichloromethylmethyldichlorosilane
Ethyltrichlorosilane
Dimethylchlorosilane
Ethylsilane
1,1,1-Trifluoropropyltrichlorosilane
Allyltrichlorosilane
Methylvinyldichlorosilane
3-Chloropropyltrichlorosilane
n-Propyltrichlorosilane
Chloromethyldimethylchlorosilane
Trimethylbromosilane
Trimethylchlorosilane
Trimethylfluorosilane
Trimethoxysilane
Cyanoethylmethyldichlorosilane
Allylmethyldichlorosilane
3-Chloropropylmethyldichlorosilane
t-Butyltrichlorosilane
Isobutyltrichlorosilane
n-Butyltrichlorosilane
Trimethylsilylisothiocyanate
n-Propylmethyldichlorosilane
Bromomethyltrimethylsilane
Ethyldimethylchlorosilane
Methoxytrimethylsilane
Dimethyldimethoxysilane
Methyltrimethoxysilane
Tetramethoxysilane
Tetramethylsilane
Diethylsilane
(N,N-Dimethylamino)dimethylsilane
1,3,5,7-Tetramethylcyclotetrasiloxane
Tetraisocyanatosilane
Cyclopentyltrichlorosilane
Trimethylsilyltrifluoroacetate
Cyclopentyldichlorosilane
n-Amyltrichlorosilane
3-Chloro-2-methylprop

In [39]:
didier12.rename(index=str, columns={'Molecule' : 'compound', 'Exp.(K)' : 'flashpoint'}, inplace=True)
didier12 = didier12[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]

In [40]:
frames = [integrated_data, didier12]
integrated_data = pd.concat(frames)
integrated_data['smiles'].unique().shape

(10461,)

In [41]:
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

In [51]:
def remove_invalid_smiles(data):
    invalid = []
    for index, row in data.iterrows():
        #print(row['smiles'])
        if Chem.MolFromSmiles(row['smiles']) == None:
            invalid.append(row['smiles'])
            #data.drop(row.index[0], inplace=True)
    print('invalid smiles strings')
    print('------------------------------')
    print(invalid)
    print('------------------------------')
    for smi in invalid:
        data.drop([data[data['smiles'] == smi].index[0]], inplace=True)

In [62]:
check_valid_smiles(integrated_data)

[]


In [55]:
integrated_data.to_csv('../data/integrated_dataset.csv', index=False)

In [210]:
integrated_data['smiles'].unique()

array(['CC(CN)O', 'C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl', 'C(CCl)Cl',
       ..., 'CCC(Cl)(CC)C(C)(C)C', 'CCCCCCC(C)(C)Cl', 'CCCCC(C)(Cl)CCC'],
      dtype=object)

## Godinho11

In [261]:
#godinho = pd.read_csv('../data/godinho11.csv')
#integrated_dataset = pd.read_csv('../data/integrated_dataset.csv')
integrated_dataset['smiles'].unique().shape
list(godinho)

['Compound', 'T FP (K)']

In [262]:
godinho['smiles'] = godinho['Compound'].apply(get_smiles)

In [263]:
godinho['source'] = 'godinho11'
godinho['pure substance'] = 1
godinho.rename(index=str, columns={'Compound' : 'compound', 'T FP (K)' : 'flashpoint'}, inplace=True)
godinho = godinho[['smiles', 'compound', 'flashpoint', 'source', 'pure substance']]
list(godinho)

['smiles', 'compound', 'flashpoint', 'source', 'pure substance']

In [264]:
godinho.head()

,smiles,compound,flashpoint,source,pure substance
0,Cc1ccccc1,toluene,280.2,godinho11,1
1,CCc1ccccc1,ethylbenzene,299.1,godinho11,1
2,Cc1ccc(C)cc1,p -xylene,298.2,godinho11,1
3,Cc1cccc(C)c1,m -xylene,298.2,godinho11,1
4,Cc1ccccc1C,o -xylene,303.2,godinho11,1


In [265]:
print(godinho.shape)
godinho.dropna(inplace=True)
print(godinho.shape)
frames = [integrated_dataset, godinho]
integrated_dataset = pd.concat(frames)
integrated_dataset['smiles'].unique().shape

(75, 5)
(75, 5)


C:\Users\Nathaniel\Miniconda3\envs\moleprop\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


(10596,)

In [267]:
integrated_dataset.to_csv('../data/integrated_dataset.csv')
#integrated_dataset[integrated_dataset['smiles'].str.contains('Si', regex=False)]
integrated_dataset.isnull().sum()

compound          0
flashpoint        0
pure substance    0
smiles            0
source            0
dtype: int64

In [250]:
integrated_dataset.drop(columns=['Compound','T FP (K)'], inplace=True)

In [254]:
integrated_dataset[integrated_dataset['source'] == np.nan]

,compound,flashpoint,pure substance,smiles,source


In [258]:
integrated_dataset.dropna(inplace=True)#['smiles'].unique().shape

In [260]:
integrated_dataset['smiles'].unique().shape

(10581,)

### HCH

In [27]:
hch = pd.read_csv('../data/hch.csv')

In [30]:
hch.isnull().sum()

compound           0
Flash point (C)    1
dtype: int64

In [29]:
hch = hch[['compound', 'Flash point (C)']]

In [31]:
hch.dropna(inplace=True)

In [44]:
hch['smiles'] = hch['compound'].apply(get_smiles)

Acetal
Acetaldehyde
Acetanilide
Acetic acid
Acetic anhydride
Acetone
Acetone cyanohydrin
Acetonitrile
Acetyl acetone
Acetyl chloride
Acetylene
Acetylene dichloride
Acetyl peroxide
Acrolein
Acrolein dimer
Acrylic acid
Acrylonitrile
Adipic acid
Adiponitrile
Aerozine 50
Aldrin
Allyl acetate
Allyl alcohol
Allyl amine
Allyl bromide
Allyl chloride
Allyl chloroformate
Allylene
Allyl glycidyl ether
2-Aminoethanol
Aminoethylethanol amine
Ammonia, anhydrous
Ammonium nitrate
n-Amyl acetate
iso-Amyl acetate
sec-Amyl acetate
n-Amyl alcohol
iso-Amyl alcohol
tert-Amyl alcohol
Amylamine
n-Amyl bromide
Amylene
n-Amyl ether
iso-Amyl formate
Amyl mercaptan
iso-Amyl nitrate
Amyl nitrite
Aniline
o-Anisaldehyde
Anisole
Anthracene
Anthraquinone
Asphalt
Aziridine
Benzaldehyde
Benzene
Benzene monochloride
Benzoic acid
p-Benzoquinone
Benzonitrile
Benzotrifluoride
Benzoyl chloride
Benzoyl peroxide
Benzyl acetate
Benzyl alcohol
Benzylamine
Benzyl benzoate
Benzyl ‘Cellosolve’
Benzyl chloride
Benzyl mercaptan
Bicyc

Pentyl alcohol
Pentyl amine
iso-Pentyl nitrite
Peracetic acid (40% acetic acid solution)
Petroleum ethers
Phenol
Phenyl acetate
Phenylcyclohexane
p-Phenylenediamine
Phenylethanolamine
Phenyl ether
Phenyl ether – Biphenyl mixture
Phenylhydrazine
o-Phenyl phenol
Phorone
iso-Phorone
Phosdrin
Phosphorus (white and yellow)
Phosphorus (red)
Phosphorus pentasulphide
Phosphorus tribromide
Phthalic anhydride
2-Picoline
4-Picoline
Picric acid
2-Pinene
Piperidine
Piperylene
1,3-Propanediamine
1,2-Propanediol
n-Propanol
Propargyl alcohol
Propargyl bromide
Propene
iso-Propenyl acetate
β-Propiolactone
Propionaldehyde
Propionic acid
Propionitrile
Propionyl chloride
iso-Propyl acetate
n-Propyl acetate
n-Propyl alcohol
iso-Propyl alcohol
Propylamine
iso-Propylamine
Propyl benzene
iso-Propyl benzene
iso-Propyl benzoate
Propyl bromide
Propyl chloride
n-Propyl cyanide
Propylene
Propylene carbonate
Propylene dichloride
iso-Propyl ether
n-Propyl formate
iso-Propyl formate
iso-Propyl glycidyl ether
Propyl ni

In [45]:
hch.isnull().sum()

compound             0
Flash point (C)      0
smiles             115
dtype: int64

In [46]:
hch.dropna(inplace=True)

In [67]:
hch.drop(index=hch[hch['Flash point (C)'].str.contains('<', regex=False)].index, inplace=True)
hch.drop(index=hch[hch['Flash point (C)'].str.contains('>', regex=False)].index, inplace=True)
hch.drop(index=hch[hch['Flash point (C)'].str.contains('\\w+-\\w', regex=True)].index, inplace=True)


In [85]:
re.compile('\\d+-\\d', re.ASCII)
value = '2–4'
print(re.search('\\d+-\\d', value, re.ASCII))
#hch
#frames = [hch, integrated_data]
#result = pd.concat(frames)

None


In [56]:
result['smiles'].unique().shape

(10631,)

In [86]:
integrated_data.isnull().sum()

compound          0
flashpoint        0
pure substance    0
smiles            0
source            0
dtype: int64